In [1]:
# imports

import os
import json
from dotenv import load_dotenv
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from agents.deals import ScrapedDeal, DealSelection